<a href="https://colab.research.google.com/github/shudaGutHub/saleem-andrew/blob/master/DMM_Daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daily MicroModels


* **LuckySeven - "LKY"**
    - LKY_numDSLR  int   (DaySinceLastRace <= 7)
    - LKY_deltaR1HDWPSR int (difference from top HDWPSR) <= 5 
    
    
    
* **Sunrise - "SUN"**


    - SUN_deltaR1HDWPSR int ( difference from top HDWPSR <= 3 )
    - SUN_rankMLO_S2LF ( rank morning line odds between 4:8) 


* **Meatloaf - "MLF"**
    
    Uses *x8_race_class* along with historical jockey, trainer data coupled with historical median 'runner_HDWPSRRating' and current 'runner_HDWPSRRating', 'race_speed_HDW_par_class_level*
    
    - MLF_isJKY: {0,1} 
    - MLF_isTRN: {0,1}
    - MLF_isHDWPSR: {0,1}
    - MLF_numHDWPAR: float 
    - MLF_numHDWPSR: float 
    
    
* **ShowMeTheMoney - "SMM"**
     
     Uses *earnings_lifetime, earnings_prevYear, earnings_curYear* to find WN bets
    
    - SMM_rankEPS_LT_L2SF: int  
    - SMM_rankEPS_CY_L2SF: int 
    - SMM_rankEPS_PY_L2SF: int 
    - SMM_rankEPS_PP_L2SF: int 
    
    
* **CarnivorousCoronavirus - "CCV"**

    Uses *days_since_last_race , num_workouts_since_last_race* to find WN bets
    - CCV_numWKSLR : int (ascending=False) more workouts is generally better
    - CCV_catWKSLR : category
    - CCV_numDSLR : int (ascending=True) less days is generally better
   
    
  
* **ThickerThanWater - "TTW"**

    Uses *pedigree_DIRT, pedigree_DIST, pedigree_MUD, pedigree_TURF* to find WN bets 
    - TTW_rankPDIRT_L2SF : int 
    - TTW_rankPDIST_L2SF : int
    - TTW_rankPMUD_L2SF : int
    - TTW_rankPTURF_L2SF : int
    - TTW_isDIRT_L2SF : int
    - TTW_isGOOD_L2SF : int

In [0]:
# wrapper for JCapper Race files - aka Past Performance (.jcp)
import pandas as pd
import numpy as np
import datetime as dt
import os
from pandas import concat, DataFrame, read_csv, to_datetime, Series, MultiIndex
from datetime import timedelta, datetime
#from horse.betsim import data
#from horse.bin.debug.simulate_vector import *
#from s3fs.core import S3FileSystem
#from horse.betsim.math import compute_probs_from_odds
from scipy.stats import entropy
from warnings import warn


In [0]:
#from horse.bin.debug.simulate_vector import *#add_residual_earnings, add_cumsum_reverse, add_runner_earnings_per_start_factors
#from horse.betsim.wrap.daily import DailyRaces
#from horse.betsim.wrap.pastperformance import PastPerformance
#from horse.betsim.wrap.jcapper import JCapper
from math import isclose

In [0]:
#fp_track_detail = "C:\\Users\\Saleem\\projects\\x8313\\data\\DMM\\track_detail.csv"
#fp_schema_pastperformance = "C:\\Users\\Saleem\\projects\\x8313\\data\\DMM\\schema_pastperformance.csv"
#fp_rebates = "C:\\Users\\Saleem\\projects\\x8313\\data\\DMM\\X8rebates.csv"

In [0]:
#def load_track_detail():
#    fp_track_detail = "C:\\Users\\Saleem\\projects\\x8313\\data\\DMM\\track_detail.csv"
#    dftd = pd.read_csv(fp_track_detail,index_col=['itsp_track_sym'])
#    return dftd
    
#def load_x8rebates():
#    fp_x8rebates =  "C:\\Users\\Saleem\\projects\\x8313\\data\\DMM\\X8rebates.csv"
#    dfx8rebates = pd.read_csv(fp_x8rebates,index_col=['itsp_track_sym'])#, usecols=['Win','Place', 'Show', 'Exacta', 'Trifecta', 'Superfecta', 'DD', 'P3', 'P4', 'P5', 'P6'])
#    dfx8rebates.drop(columns=cols_prefix(dfx8rebates,'Unnamed'), axis=1, inplace=True)
#    #df['is_x8_track'] = df['itsp_track_sym'].isin(dfx8rebates.index.values).astype(int)
#    #cols_rebate = ['rebate_pct_{}' + ]
#    return dfx8rebates.drop(['Track'], axis=1)/100.0

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#os.chdir('/content/drive/My Drive/Ben/DMM/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.getcwd()

'/content/drive/My Drive/Ben/DMM'

A wrapper and pipeline function to track each transformation and cleaning step

In [0]:
def loggg(f):
    """A decorator function to wrap data pipeline"""
    def wrapper(df, *args, **kwargs):
        tic = dt.datetime.now()
        print(f"{f.__name__} uses {*args, *kwargs}")
        result = f(df, *args, **kwargs)
        toc = dt.datetime.now()
        print(f"{f.__name__} took {toc - tic} shape={result.shape}")
        return result
    return wrapper

In [0]:
@loggg
def start_pipeline(df):
    return df.copy()

In [0]:
@loggg
def add_num_nulls_race_by_attr(df, attr):
    """df: df with 'race_id' 
    attr: any attribute to count nulls by race
    groups by race_id and counts nulls"""
    df['num_null_{}'.format(attr)] = df.groupby('race_id')[attr].transform(lambda x:x.isnull().sum()).astype(int)
    print("added column num_null_{}".format(attr))
    return df

In [0]:
@loggg
def get_clean_races_attr(df, attr):
    """Takes races with no scratches
    and no null HDWPSR"""
    df = df[df['num_null_{}'.format(attr)]==0]
    return df
@loggg
def get_clean_races_scratch(df):
    """Gets only races where num_starters_pre == num_starters_post
    and num_starters_starters gt than min_starters"""
    df=df[df.num_starters_pre == df.num_starters_post]
    return df


In [0]:
@loggg
def add_rank_L2SF(df, attr):
    """ranks L2SF : Largest2Smallest tiebreak 'first' 
    Assumes sorted by post_position 1,2,3,.."""
    #df = df.groupby('race_id', as_index=False).apply(lambda df: df.sort_values(by=by))
    df['rank_{}_L2S'.format(attr)] = df.groupby('race_id')[attr].transform(
        lambda x: x.rank(ascending=False, method='first'))
    print('added col: rank_{}_L2S'.format(attr))
    return df

@loggg
def add_DiffFromTo(df, attr, ascending):
    """
    The difference between the R1 and the 
    returns df[attr].sort_values(ascending=False).diff(axis=0)
    
       """
    df['DiffFromTo_{}'.format(attr)] = df.groupby('race_id')[attr].transform(lambda x:x.sort_values(ascending=ascending).diff().cumsum().abs())
    return df

In [0]:
@loggg
def convert_dates(df):
    cols_pp_dates = cols_prefix(df, 'pp_date')
    cols_wk_dates = cols_prefix(df, 'wk_date')
    cols_other_dates = ['date', 'runner_birthdate_year', 'runner_birthdate_act']
    cols_dates = cols_pp_dates + cols_wk_dates + cols_other_dates
    df[cols_dates] = df[cols_dates].applymap(pd.to_datetime)
    return df

In [0]:
@loggg
def clean_cols_TTW(df):
    """Replace 9999 values with 0. for"""
    cols_PED = ['runner_pedigree_HDWDirt',
     'runner_pedigree_HDWMud',
     'runner_pedigree_HDWTurf',
     'runner_pedigree_HDWDist']
    df[cols_PED] = df[cols_PED].fillna(0)
    df[cols_PED] = df[cols_PED].applymap(lambda x: x if x<9999 else 0)
    return df

@loggg
def add_columns_TTW(df):
    df['race_surface'] = df['race_surface'].astype('category')
    df['TTW_isDIRT'] = df['race_surface'].isin(['D','d']).astype(int)
    if 'track_condition_code' in cols_prefix(df,'track_condition_code'):
        df['TTW_isGOOD'] = df['track_condition_code'].isin(['FST','FM','GD']).astype(int)
        df['cat_TKCOND'] = df['track_condition_code'].astype('category')
    df['is_NEGDIST'] = df['race_distance'].map(lambda x:x<0).astype(int)
    df['numDIST'] = df['race_distance'].map(lambda x:abs(x))
    return df
@loggg
def add_TTW_rscore(df):
    cols_rank_PED = cols_prefix(df, 'rank_runner')
    cols_rank1 = ['isR1_'+ c  for c in cols_rank_PED]
    df[cols_rank1] = df[cols_rank_PED].applymap(lambda x:int(x==1))
    df['score_R1'] = df[cols_rank1].sum(axis=1)
    return df



In [0]:
@loggg
def add_wk_days_since_lastrace(df, col_date_pp='pp_date_0'):
    """
    for each workout date subtract date from 'pp_date_0' or other reference date column
    
    
    """
    #df[col_date_pp] = df[col_date_pp].map(pd.to_datetime) #Convert last race date to datetime
    
    gen_names_wk = lambda x: [x + '_' + str(wknum) for wknum in range(1, 13)]
    cols_wkdate = gen_names_wk('wk_date')  #wk_date_1 , wk_date_2, ...wk_date_10
    print(cols_wkdate)
    df[cols_wkdate] = df[cols_wkdate].applymap(pd.to_datetime)
    
    dfdays = df[gen_names_wk('wk_date')]-df[col_date_pp].values[:,None]
    print(dfdays.head())
    colnames_days_since_last_race_date = gen_names_wk('wk_days_since_last_race')
    df[colnames_days_since_last_race_date] = dfdays  #add into original frame   
    #dfdays = pd.concat([df.apply(lambda dfrow: dfrow[wkcol]-dfrow[col_date_pp], axis=1) for wkcol in cols_wk_date] ,axis=1)
    #number of days between workout date and last race
    #pd.concat([dfAQU.apply(lambda dfrow: dfrow[wkcol]-dfrow[col_date_pp], axis=1) for wkcol in cols_wk_dates] ,axis=1)
    return df


@loggg
def add_num_wk_since_lastrace(df):
    cols_wk_days_since_last_race = cols_prefix(df, 'wk_days_since_last_race')
    assert len(cols_wk_days_since_last_race) ==12
    colnames_wk_is_since_last_race = gen_names_wk('wk_is_since_last_race')
    df[colnames_wk_is_since_last_race] = df[cols_wk_days_since_last_race].applymap(lambda x:int(x.days>0))
    df[colnames_wk_is_since_last_race].head()
    df['num_wk_since_last_race'] = df[colnames_wk_is_since_last_race].sum(axis=1) 
    return df

@loggg
def add_bin7_days_since_last_race(df, step, col_numDSLR):
    df['bin_days_since_last_race'] = pd.cut(df.loc[:,col_numDSLR], bins = np.arange(0,2000,step))
    return df
@loggg
def add_qbin4_days_since_last_race(df):
    df['qbin4'] = pd.qcut(df.days_since_last_race,4)
    return df
@loggg
def add_btn_lost_last_3(df):
    """(btn_0 + btn_1 + btn_2)^2 / (btn_0 + btn_1 + btn_2)^2 + (lost_0+lost_1+lost_2)^2 """
    cols_btn = cols_prefix(df, 'x8btn')
    cols_lost = cols_prefix(df, 'x8lost')
    df['btn_last_3'] = df[cols_btn].sum(axis=1)
    df['lost_last_3'] = (df[cols_lost].sum(axis=1))
    df['ratio_btnlost_3'] = np.power(df['btn_last_3'],2) / (np.power(df['btn_last_3'],2) + np.power(df['lost_last_3'],2))
    #df['btn_lost_sq_ratio'] = df['btn_last_3']*df['btn_last_3'] / 'lost_last_3']].apply(lambda b,l : np.power(b,2)/((np.power(b,2) + np.power(l,2))), axis=1)
    print(df['ratio_btnlost_3']).head()
    return df
    

@loggg
def add_NWclass_by_epoch(df, epoch):
    """ Adds NW eligibiily by epoch
    'lifetime', 'currentYear', 'prevYear'"""
    map_NWclass = {0 : 'M',
                  1 : 'NW2',
                  2: 'NW3',
                  3: 'NW4'}
    # bucket wine quality scores into qualitative quality labels
    df['NW_class_{}'.format(epoch)] = df['runner_horse_{}_wins'.format(epoch)].map(lambda wins: map_NWclass.get(wins, 'GT3'))
    return df
    

@loggg
def add_is_win_last_race(df, col_lastracefinish):
    """
    df: DataFrame
    col_finish_last: 'pp_call_finish_pos_0' if using pp files
                     'last_race_finish' if using result files  """
    
    df['isWLR'] = df[col_lastracefinish].map(lambda x: int(x==1))
    return df

@loggg
def add_num_winners_last_race(df, col_isWLR):
    df['numWLR'] = df.groupby('race_id')[col_isWLR].transform(sum)
    return df




@loggg
def add_residual_earnings(df):
    df.loc[:,'runner_horse_residual_earnings'] = df.loc[:, 'runner_horse_lifetime_earnings'] -  df.loc[:, 'runner_horse_currentYear_earnings'] - df.loc[:, 'runner_horse_prevYear_earnings']
    df.loc[:,'runner_horse_residual_starts'] = df.loc[:, 'runner_horse_lifetime_starts'] -  df.loc[:, 'runner_horse_currentYear_starts'] - df.loc[:, 'runner_horse_prevYear_starts']
    return df
@loggg
def add_runner_earnings_per_start_factors(df):
    """computes earnings per start for a given group """
    df.loc[:,'eps_lifetime'] = (df.loc[:,'runner_horse_lifetime_earnings']/df.loc[:,'runner_horse_lifetime_starts']).fillna(0)
    df.loc[:,'eps_currentYear'] = (df.loc[:,'runner_horse_currentYear_earnings']/df.loc[:,'runner_horse_currentYear_starts']).fillna(0)
    df.loc[:,'eps_prevYear'] = (df.loc[:,'runner_horse_prevYear_earnings']/df.loc[:,'runner_horse_prevYear_starts']).fillna(0)
    df.loc[:,'eps_distance'] = (df.loc[:,'runner_horse_lifetime_earnings_distance']/df.loc[:,'runner_horse_lifetime_starts_distance']).fillna(0)
    df.loc[:,'eps_residual'] = (df.loc[:,'runner_horse_residual_earnings']/df.loc[:,'runner_horse_residual_starts']).fillna(0)
    #df.loc[:,'eps_ppstarts'] = (df.loc[:,'runner_horse_earnings_pp_total']/df.loc[:,'num_starts_from_pp_earnings']).fillna(0)

    #df.loc[:,'eps_pp_total'] = (df.loc[:,'runner_horse_earnings_pp_total']/df.loc[:,'runner_horse_earnings_']).fillna(0)
    return df

@loggg
def add_rank_eps(df):
    df['rank_eps_lifetime'] = df.groupby('race_id')['eps_lifetime'].transform(lambda x: x.rank(method='first',ascending=False))
    df['rank_eps_currentYear'] = df.groupby('race_id')['eps_currentYear'].transform(lambda x: x.rank(method='first',ascending=False))
    df['rank_eps_prevYear'] = df.groupby('race_id')['eps_prevYear'].transform(lambda x: x.rank(method='first',ascending=False))
    df['rank_eps_distance'] = df.groupby('race_id')['eps_distance'].transform(lambda x: x.rank(method='first',ascending=False))
    df['rank_eps_residual'] = df.groupby('race_id')['eps_residual'].transform(lambda x: x.rank(method='first',ascending=False))
    #df['rank_eps_ppstarts'] = df.groupby(level='race_id')['eps_ppstarts'].transform(lambda x: x.rank(method='first',ascending=False))

    return df

@loggg
def add_quality_num_scratches(df, col_pre, col_post):
    df['num_scratches'] = (df.loc[:,'num_starters_pre_jcp'].fillna(0) - df.loc[:,'num_starters_post_jcp'].fillna(0)).astype(int)
    df['quality_scratch'] = df['num_scratches'].apply(lambda value: 'clean' if value == 0 else
                                                     'single' if value == 1 else
                                                     'dirty')

@loggg
def add_quality_HDWPSR(df):
    df['quality_label_race_HDWPSR'] = df['num_isnull_hdw'].apply(lambda value: 'clean' if value == 0 else 'dirty')
    df['quality_HDWPSR'] = pd.Categorical(df['quality_label_race_HDWPSR'], categories=['clean','dirty'])
    return df

@loggg
def add_quality_num_scratches(df):
    col_pre = cols_prefix('num_starters_pre')[0]
    col_post = cols_prefix('num_starters_post')[0]

    df['num_scratches'] = (df.loc[:, cols_pre].fillna(0) - df.loc[:,col_post].fillna(0)).astype(int)
    df['quality_scratch'] = df['num_scratches'].apply(lambda value: 'clean' if value == 0 else
                                                     'single' if value == 1 else
                                                     'dirty')
    return df

@loggg
def add_pp_beaten(df):
    """ for each pp_race determine how many horses were beaten"""
    cols_btn = ['ppx8_beaten_{}'.format(pp) for pp in range(10)]
    cols_pp_field = cols_prefix(df, 'pp_fieldsize')
    cols_pp_finish = cols_prefix(df, 'pp_call_finish_pos')
    df[cols_btn] = np.subtract(df[cols_pp_field],df[cols_pp_finish])
    return df
@loggg
def add_pp_lost(df):
    """ for each pp_race determine how many horses were lost to """
    cols_lost = ['ppx8_lost_{}'.format(pp) for pp in range(10)]
    cols_btn = cols_prefix(df, 'ppx8_beaten')
    cols_pp_field = cols_prefix(df, 'pp_fieldsize')
    cols_pp_finish = cols_prefix(df, 'pp_call_finish_pos')
    df[cols_lost] = np.subtract(df[cols_pp_field],df[cols_btn] + 1)
    return df

@loggg
def add_pp_pct_beaten(df):
    """ for each pp_race compute pct of field beaten """
    cols_pp_pct_beaten = ['ppx8_pct_beaten_{}'.format(pp) for pp in range(10)]
    cols_btn = cols_prefix(df, 'ppx8_beaten')
    cols_pp_field = cols_prefix(df, 'pp_fieldsize')
    #cols_pp_finish = cols_prefix(df, 'pp_call_finish_pos')
    df[cols_pp_pct_beaten] = np.divide(df[cols_btn],df[cols_pp_field])
    return df

@loggg
def add_pp_dollars_per_runner(df):
    """for reach pp race compute the dollars per runner """
    cols_dollars_per_runner = ['x8dpr_{}'.format(pp) for pp in range(10)]  
    cols_pp_purse = cols_prefix(df,'pp_purse')
    cols_pp_field = cols_prefix(df,'pp_field')
    df[cols_dollars_per_runner] = np.divide(df[cols_pp_purse],df[cols_pp_field])
    return df
    
@loggg
def add_pp_dollars_beaten(df):
    cols_dollars_per_runner = cols_prefix(df,'x8dpr_')
    cols_btn = cols_prefix(df,'ppx8_beaten_')
    cols_dollars_beaten = ['ppx8_dollarsBeaten_{}'.format(pp) for pp in range(10)]
    df[cols_dollars_beaten] = np.multiply(df[cols_btn], df[cols_dollars_per_runner])
    return df

@loggg
def add_mean_dollars_beaten_lastN(df, n):
    cols_dollars_beaten = cols_prefix(df,'ppx8_dollarsBeaten_')
    #df['sum_ppx8_dollars_beaten_last3'] = df[cols_dollars_beaten].iloc[:,0:n].sum(axis=1)
    df['mean_ppx8_dollars_beaten_last3'] = df[cols_dollars_beaten].iloc[:,0:n].mean(axis=1)
    return df

@loggg
def add_sum_dollars_beaten_lastN(df, n):
    cols_dollars_beaten = cols_prefix(df,'ppx8_dollarsBeaten_')
    #df['sum_ppx8_dollars_beaten_last3'] = df[cols_dollars_beaten].iloc[:,0:n].sum(axis=1)
    df['sum_ppx8_dollars_beaten_last3'] = df[cols_dollars_beaten].iloc[:,0:n].sum(axis=1)
    return df

### Some convenience functions for finding and naming columns

In [0]:
def clean_columns(df):
    """
    Removes '_pp' from pp columns

    """
    df.columns = list(map(lambda x: x.replace('_pp', ''), df.columns))
    df.drop(cols_prefix(df, 'wager_'), inplace=True, axis=1)
    df.drop(cols_prefix(df, 'race_reserved'), inplace=True, axis=1)
    df.drop(cols_prefix(df, 'race_blank'), inplace=True, axis=1)
    df.drop(cols_prefix(df, 'pp_reserved'), inplace=True, axis=1)
    return df

In [0]:

cols_prefix = lambda df, txt: [c for c in df.columns if c.startswith(txt)]
cols_suffix = lambda df, txt: [c for c in df.columns if c.endswith(txt)]

vals_around = lambda df, cols, thresh, around: df[cols].applymap(lambda x: abs(x - thresh) < around)
# Column name generator
gen_names_wk = lambda x: [x + '_' + str(wknum) for wknum in range(1, 13)]

# generating column names
def gen_ppnames_dr(factor='speed_HDW', prefix='pp', suffix=None):
    """
    generates a list of pp column names for reading or adding
    i.e. if I know the factor name I can generate the column names
    i.e. if I want to add columns I can generate the new names in a similar format
    :param factor: 'speed_HDW' (a factor_name from DailyRaces pp  ie. 'speed_HDW)
    :param prefix: 'pp'
    :param suffix: 'pp'
    :return: ['pp_speed_HDW_0_pp', 'pp_speed_HDW_1_pp', ..., pp_speed_HDW_9_pp']
    """
    if (suffix):
        namespp = [prefix + "_" + factor + "_" + str(pp) + "_" + suffix for pp in range(10)]
    else:
        namespp = [prefix + "_" + factor + "_" + str(pp) for pp in range(10)]

    return namespp


# Load Historical PastPerformance DataFrame 

Load the historical dataframe

In [0]:
## The output from current loader is in this file
dfhist = pd.read_csv('dfhist_DMM.csv.gz',low_memory=False)


### Start the pipeline and generate columns for TTW model

In [0]:
#Start the pipeline to clean the data

dfhist_clean = start_pipeline(dfhist)\
.pipe(add_num_nulls_race_by_attr,'runner_HDWPSRRating')\
.pipe(get_clean_races_attr, 'runner_HDWPSRRating')\
.pipe(clean_cols_TTW)\
.pipe(add_rank_L2SF,'runner_pedigree_HDWDirt')\
.pipe(add_rank_L2SF, 'runner_pedigree_HDWDist')\
.pipe(add_rank_L2SF, 'runner_pedigree_HDWMud')\
.pipe(add_rank_L2SF, 'runner_pedigree_HDWTurf')\
.pipe(add_rank_L2SF, 'runner_HDWPSRRating')\
.pipe(add_columns_TTW)\
.pipe(add_TTW_rscore)\
.pipe(add_DiffFromTo, attr='runner_HDWPSRRating', ascending=False)

start_pipeline uses ()
start_pipeline took 0:00:00.268273 shape=(24550, 1456)
add_num_nulls_race_by_attr uses ('runner_HDWPSRRating',)
added column num_null_runner_HDWPSRRating
add_num_nulls_race_by_attr took 0:00:01.490967 shape=(24550, 1457)
get_clean_races_attr uses ('runner_HDWPSRRating',)
get_clean_races_attr took 0:00:00.130815 shape=(19750, 1457)
clean_cols_TTW uses ()
clean_cols_TTW took 0:00:00.032985 shape=(19750, 1457)
add_rank_L2SF uses ('runner_pedigree_HDWDirt',)
added col: rank_runner_pedigree_HDWDirt_L2S
add_rank_L2SF took 0:00:00.979137 shape=(19750, 1458)
add_rank_L2SF uses ('runner_pedigree_HDWDist',)
added col: rank_runner_pedigree_HDWDist_L2S
add_rank_L2SF took 0:00:01.201124 shape=(19750, 1459)
add_rank_L2SF uses ('runner_pedigree_HDWMud',)
added col: rank_runner_pedigree_HDWMud_L2S
add_rank_L2SF took 0:00:00.998985 shape=(19750, 1460)
add_rank_L2SF uses ('runner_pedigree_HDWTurf',)
added col: rank_runner_pedigree_HDWTurf_L2S
add_rank_L2SF took 0:00:01.074353 shap

# 

### Now continue the pipeline adding columns for SMM, CCV

In [0]:

#Cleaning pipeline history all tracks
dfhist_clean = start_pipeline(dfhist_clean)\
.pipe(clean_columns)\
.pipe(convert_dates)\
.pipe(add_wk_days_since_lastrace)\
.pipe(add_num_wk_since_lastrace)\
.pipe(add_bin7_days_since_last_race, step=7, col_numDSLR='days_since_last_race')\
.pipe(add_NWclass_by_epoch, 'lifetime')\
.pipe(add_NWclass_by_epoch, 'currentYear')\
.pipe(add_NWclass_by_epoch, 'prevYear')\
.pipe(add_is_win_last_race, col_lastracefinish='pp_call_finish_pos_0')\
.pipe(add_rank_L2SF, 'runner_HDWPSRRating')\
.pipe(add_residual_earnings)\
.pipe(add_runner_earnings_per_start_factors)\
.pipe(add_rank_eps)\
.pipe(add_pp_beaten)\
.pipe(add_pp_lost)\
.pipe(add_pp_pct_beaten)\
.pipe(add_pp_dollars_per_runner)\
.pipe(add_pp_dollars_beaten)\
.pipe(add_mean_dollars_beaten_lastN,3)\
.pipe(add_sum_dollars_beaten_lastN,3)

start_pipeline uses ()
start_pipeline took 0:00:00.181002 shape=(19750, 1472)
convert_dates uses ()
convert_dates took 0:00:58.115775 shape=(19750, 1326)
add_wk_days_since_lastrace uses ()
['wk_date_1', 'wk_date_2', 'wk_date_3', 'wk_date_4', 'wk_date_5', 'wk_date_6', 'wk_date_7', 'wk_date_8', 'wk_date_9', 'wk_date_10', 'wk_date_11', 'wk_date_12']
  wk_date_1 wk_date_2 wk_date_3  ... wk_date_10 wk_date_11 wk_date_12
0   21 days  -35 days  -85 days  ...  -312 days  -312 days  -312 days
1   29 days   21 days   13 days  ...  -167 days  -167 days  -167 days
2  -16 days  -48 days  -78 days  ...  -295 days  -295 days  -295 days
3   15 days   -4 days  -40 days  ...  -257 days  -257 days  -257 days
4   20 days   -2 days   -9 days  ...   -69 days   -69 days   -69 days

[5 rows x 12 columns]
add_wk_days_since_lastrace took 0:00:01.191080 shape=(19750, 1338)
add_num_wk_since_lastrace uses ()
add_num_wk_since_lastrace took 0:00:02.207284 shape=(19750, 1351)
add_bin7_days_since_last_race uses ('step

In [0]:
dfhist_clean.to_csv('dfhist_clean.csv',index=False)